### Configuración de librerías

In [1]:
# Manejo de dataset y cálculos numéricos
import pandas as pd
import requests
import json
import numpy as np
import time

from datetime import datetime
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ml_amazon")

# Recopilación de datos
A partir de este punto se reúnen todos los datos necesarios en caso de no tenerlos ya, para poder entrenar el modelo y realizar predicciones. Estos datos son los siguientes:

+ Ordenes: Es el set de datos proporcionado por el cliente, con toda la información de las ventas realizadas. La información clave que vamos a extraer es:
    - Fecha
    - Ciudad
    - Producto
    - Cantidad vendida
    - Precio de la unidad

+ Clima: Nos da la información meteorológica de las ciudades más importantes donde vende el cliente:
    - Fecha
    - Ciudad
    - Velocidad del viento
    - Precipitaciones
    - Nieve
    - Profundidad de la nieve
    - Temperatura media

## Lectura dataset de ordenes

In [4]:
df_orders = pd.read_excel('ordenes(01-01-2019_29_08_2021).xlsx', index_col=0)
df_orders = df_orders.loc[:, ['purchase-date','ship-city','asin','quantity','item-price']]
df_orders.dropna(inplace=True)
new_date = []
for date in df_orders['purchase-date']:
    new_date.append(datetime.strptime(date, '%Y-%m-%dT%H:%M:%S+00:00').strftime('%Y-%m-%d'))
df_orders['purchase-date'] = new_date
cities = []
for city in df_orders['ship-city']:
    cities.append(city.title())
df_orders['ship-city'] = cities
df_orders = df_orders.groupby(['purchase-date','ship-city','asin']).sum().reset_index()
df_orders.rename(columns={'purchase-date':'DATE','ship-city':'CITY','asin':'PRODUCT','quantity':'QUANTITY','item-price':'PRICE'},
               inplace=True)
df_orders['DATE'] = pd.to_datetime(df_orders.DATE)
df_orders = df_orders.sort_values('DATE',ascending=True)
df_orders.head()

,DATE,CITY,PRODUCT,QUANTITY,PRICE
0,2019-01-01,Cincinnati,B00B4S6SLW,1.0,99.00
13,2019-01-01,Stuttgart,B00NM9HO3W,1.0,36.00
12,2019-01-01,Slough,B07JP2ZRT9,1.0,69.99
10,2019-01-01,San Fernando,B07CMJ5186,1.0,55.00
9,2019-01-01,New Braunfels,B00PUI9WKG,1.0,30.00


Comprobamos cuantas ciudades hay, además de cuantas veces aparece cada una (cuanto más aparezcan más compras se realizan desde allí), esto lo guardamos en un excel, para tenerlo por si acaso fuera necesario.

In [ ]:
# En all_cities tendremos todas las ciudades sin duplicados
all_cities = df_orders['CITY'].drop_duplicates()

#Guardamos en un excel cuantas veces se repite cada ciudad
num_all_cities = all_cities.value_counts().to_frame()
num_all_cities.to_excel('num_all_cities.xlsx')

In [ ]:
# Número total de ciudades
cities = list(set(all_cities))
print("Tenemos un total de " + str(len(cities)) + " ciudades.")

### Mediante Geolocator obtenemos geolocalización de las ciudades

In [ ]:
df_locations = pd.DataFrame([], columns = ['CITY' , 'LATITUDE', 'LONGITUDE'])
latitude = []
longitude = []
altitude = []
i=0
for city in all_cities:
    location = geolocator.geocode(city)
    if location is not None:
        df_locations.loc[i]=[ city, round(location.latitude), round(location.longitude) ]
        print("Ciudad: " + city + " i: " + str(i))
        i = i + 1

In [ ]:
# Guardamos en un excel cada ciudad con sus coordenadas
df_locations.to_excel('all_cities.xlsx', index=False)
df_locations.head()

### Leemos el dataset que contiene cada ciudad con sus coordenadas

In [ ]:
df_locations = pd.read_excel('all_cities.xlsx')
df_locations = df_locations[['CITY','LATITUDE','LONGITUDE']]
df_locations['LATITUDE'] = df_locations['LATITUDE'].astype(float)
df_locations['LONGITUDE'] = df_locations['LONGITUDE'].astype(float)
df_locations.head()

### df_stations: Lectura dataset con datos de las estaciones de clima disponibles

In [ ]:
df_stations = pd.read_fwf('stations_info_USA.txt')
df_stations = df_stations.drop(['ELE__N','CODE','NUMBER'], 1)
df_stations.dropna()
df_stations.rename(columns={'STATIONS__N':'STATIONS',
                            'LAT___N':'LATITUDE',
                            'LONG___N':'LONGITUDE',
                            'ST':'STATE',
                            'NAME_________________________M':'NAME'},
               inplace=True)
df_stations['LATITUDE'] = round(df_stations['LATITUDE'])
df_stations['LONGITUDE'] = round(df_stations['LONGITUDE'])
df_stations.to_excel('stations_info_USA.xlsx', index=False)
df_stations

### df_stations_merged: Ciudad y estación con coordenadas

In [ ]:
df_stations_merged = pd.merge(df_stations, df_locations, left_on=['LATITUDE','LONGITUDE'], right_on=['LATITUDE','LONGITUDE'], how='right')
df_stations_merged = df_stations_merged[['STATIONS','LATITUDE','LONGITUDE','CITY','NAME']]
df_stations_merged.drop_duplicates()
df_stations_merged.dropna(inplace=True)
df_stations_merged

In [ ]:
station_list = df_stations_merged['STATIONS'].drop_duplicates()
stations = list(set(df_stations_merged['STATIONS']))
print("Tenemos un total de " + str(len(stations)) + " estaciones climáticas.")

### Mediante API obtenemos los datos climáticos de cada estación (10 horas de ejecución)

In [ ]:
df_weather_data = pd.DataFrame()
i=0
for station in station_list:
    url = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=NAME,AWND,PRCP,SNOW,SNWD,TAVG&stations="+station+"&startDate=2019-01-01&endDate=2021-08-29&units=metric&includeStationLocation=1&format=json"
    ## try catch y pass para que pase
    try:
        response = requests.get(url)
        data = json.loads(response.content)
        if data is not None:
            df_data = pd.json_normalize(data)
            df_data.dropna(inplace=True)
            df_weather_data = pd.concat([df_weather_data, df_data])
            df_weather_data.dropna(inplace=True)
            i=i+1
            print("Station: " + station + ", i: " + str(i))
    except:
        print("Something went wrong")
    finally:
        df_weather_data.to_excel('weather_data_USA.xlsx', index=False)

In [ ]:
df_weather_data['LATITUDE'] = round(df_weather_data['LATITUDE'])
df_weather_data['LONGITUDE'] = round(df_weather_data['LONGITUDE'])
df_weather_merged = pd.merge(df_weather_data, df_locations, left_on=['LATITUDE','LONGITUDE'], right_on=['LATITUDE','LONGITUDE'], how='right')
df_weather_merged.dropna(inplace=True)
df_weather_merged.rename(columns={'PRCP':'PRECIPITATION',
                            'SNWD':'SNOW_DEPTH',
                            'TAVG':'TEMPERATURE',
                            'AWND':'SPEED_WIND'},
               inplace=True)
df_weather_merged = df_weather_merged[['CITY','DATE','SPEED_WIND','PRECIPITATION','SNOW','SNOW_DEPTH','TEMPERATURE']]
df_weather_merged = df_weather_merged.groupby(['DATE','CITY']).mean().reset_index()
cities = []
for city in df_weather_merged['CITY']:
    cities.append(city.title())
df_weather_merged['CITY'] = cities
df_weather_merged['DATE'] = pd.to_datetime(df_weather_merged.DATE)
df_weather_merged.head()

# Unión de los datos con las ordenes y climas (Definitivo)

In [5]:
df_merged = pd.merge(df_orders, df_weather_merged, left_on=['DATE','CITY'], right_on=['DATE','CITY'], how='inner').reset_index()
df_merged = df_merged[['DATE','CITY','PRODUCT','QUANTITY','PRICE','TEMPERATURE','SPEED_WIND','PRECIPITATION','SNOW','SNOW_DEPTH']]
df_merged = df_merged.sort_values('DATE',ascending=True)
df_merged

,DATE,CITY,PRODUCT,QUANTITY,PRICE,TEMPERATURE,SPEED_WIND,PRECIPITATION,SNOW,SNOW_DEPTH
0,2019-01-01,Cincinnati,B00B4S6SLW,1.0,99.00,8.500000,3.600000,0.00,0.0,0.0
1,2019-01-01,New Braunfels,B00PUI9WKG,1.0,30.00,6.850000,5.000000,0.00,0.0,0.0
2,2019-01-01,Duluth,B07GMJB5ZZ,1.0,60.00,-18.900000,3.100000,0.00,0.0,360.0
3,2019-01-01,Denver,B07GMJB5ZZ,1.0,60.00,-15.500000,2.300000,0.00,0.0,0.0
13,2019-01-02,Dallas,B01L9SOIHM,1.0,45.00,1.500000,4.400000,19.30,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6132,2021-08-29,Chicago,B06Y44DZC4,1.0,39.99,27.500000,4.700000,7.10,0.0,0.0
6127,2021-08-29,Miami,B00B4S6SLW,1.0,97.00,29.600000,4.050000,0.25,0.0,0.0
6126,2021-08-29,Larchmont,B07CMJ5186,1.0,59.99,21.766667,2.666667,0.00,0.0,0.0
6128,2021-08-29,New York,B07CMJ5186,1.0,59.99,21.766667,2.666667,0.00,0.0,0.0


In [7]:
df_merged.to_excel('weather_city_orders.xlsx', index=False)